<a href="https://colab.research.google.com/github/danielhcg/AntiSmishGPT/blob/main/gemini_process_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import time
import requests

# Load the spreadsheet
from google.colab import files
uploaded = files.upload()
# Get the first filename from the uploaded dictionary
file = next(iter(uploaded))
df = pd.read_excel(file, header=None)

# Function to process a batch of messages
def process_batch(messages):
    payload = {
        "prompt": "Is this message real or fake?",
        "messages": messages
    }
    headers = {
        "Authorization": "GOOGLE_API_KEY",  # Replace with your actual API key
        "Content-Type": "application/json"
    }
    response = requests.post("https://generativelanguage.googleapis.com", json=payload, headers=headers)
    return response.json()

# Divide the rows into chunks of 120
chunk_size = 120
chunks = [df[i:i + chunk_size] for i in range(0, df.shape[0], chunk_size)]

# Process each chunk with rate limits
results = []
requests_made = 0

for i, chunk in enumerate(chunks):
    messages = chunk[0].tolist()  # Assuming each row is a message
    result = process_batch(messages)
    results.append(result)

    requests_made += 1

    # Rate limiting: ensure only 2 requests per minute
    if requests_made % 2 == 0:
        time.sleep(60)

    # Stop execution once daily limit is reached
    if requests_made == 50:
        print("Daily limit reached. Stopping execution.")
        break

# Process results to extract "Real", "Fake", or "Undetermined"
processed_results = []
for chunk, result in zip(chunks[:requests_made], results):
    for message, analysis in zip(chunk[0], result['responses']):
        status = analysis.get('status', 'Undetermined')  # Assuming API response includes a 'status' field
        processed_results.append({'message': message, 'status': status})

# Convert to DataFrame and save to a new file
results_df = pd.DataFrame(processed_results)
results_df.to_excel('/mnt/data/processed_results.xlsx', index=False)

print("Processing complete. Results saved to 'processed_results.xlsx'.")


Saving mendeley_set.xlsx to mendeley_set.xlsx


JSONDecodeError: Expecting value: line 1 column 1 (char 0)